<a href="https://colab.research.google.com/github/OluwatodimuAdegoke/MNIST/blob/main/ASL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This model is designed to identify different American Sign Language alphabet using the ASL Dataset from Kaggle

In [26]:
#Using a ASL datasets from kaggle
import pandas as pd
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Dense,
    Conv2D,
    MaxPool2D,
    Flatten,
    Dropout,
    BatchNormalization,
)


train_df = pd.read_csv("data/sign_mnist_train.csv")
valid_df = pd.read_csv("data/sign_mnist_valid.csv")

In [27]:
#Preparing the model
y_train = train_df['label']
y_valid = valid_df['label']
del train_df['label']
del valid_df['label']

x_train = train_df.values
x_valid = valid_df.values


In [8]:
#Normalize x_train and x_valid.
x_train = x_train/255
x_valid = x_valid/255

In [28]:
num_classes = 24

#Categorically encoding y_train and y_valid.

y_train = keras.utils.to_categorical(y_train,num_classes)
y_valid = keras.utils.to_categorical(y_valid,num_classes)


x_train = x_train.reshape(-1,28,28,1)
x_valid = x_valid.reshape(-1,28,28,1)

x_train.shape, x_valid.shape

((27455, 28, 28, 1), (7172, 28, 28, 1))

In [29]:
#Building the model
model = Sequential()
model.add(Conv2D(75, (3, 3), strides=1, padding="same", activation="relu",
                 input_shape=(28, 28, 1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(50, (3, 3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Conv2D(25, (3, 3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2, 2), strides=2, padding="same"))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(units=num_classes, activation="softmax"))


In [33]:
#Adding Data Augumentation to increase the accuracy.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    zoom_range=0.1,  # Randomly zoom image
    width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images horizontally
    vertical_flip=False, # Don't randomly flip images vertically
)

batch_size = 32
img_iter = datagen.flow(x_train, y_train, batch_size=batch_size)

datagen.fit(x_train)

The initial network used had a high accuracy but the model was overfitting the data.

Conclusion: After adding more layers to the model, the accuracy became 98%, which solved the overfitting discovered in the previous model.

In [34]:
#Compiling the model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'])
#Training the model for 20 epochs.
model.fit(img_iter,
          epochs=10,
          steps_per_epoch=len(x_train)/batch_size, # Run same number of steps we would if we were not using a generator.
          validation_data=(x_valid, y_valid))

Epoch 1/10
857/857 [==============================] - 111s 128ms/step - loss: 0.8317 - accuracy: 0.7928 - val_loss: 0.4063 - val_accuracy: 0.8717
Epoch 2/10
857/857 [==============================] - 108s 125ms/step - loss: 0.2708 - accuracy: 0.9117 - val_loss: 0.1152 - val_accuracy: 0.9593
Epoch 3/10
857/857 [==============================] - 112s 130ms/step - loss: 0.1822 - accuracy: 0.9415 - val_loss: 0.4591 - val_accuracy: 0.8703
Epoch 4/10
857/857 [==============================] - 115s 134ms/step - loss: 0.1485 - accuracy: 0.9518 - val_loss: 0.1344 - val_accuracy: 0.9584
Epoch 5/10
857/857 [==============================] - 108s 126ms/step - loss: 0.1296 - accuracy: 0.9594 - val_loss: 0.1060 - val_accuracy: 0.9633
Epoch 6/10
857/857 [==============================] - 111s 130ms/step - loss: 0.1050 - accuracy: 0.9666 - val_loss: 0.1584 - val_accuracy: 0.9520
Epoch 7/10
857/857 [==============================] - 110s 128ms/step - loss: 0.1055 - accuracy: 0.9682 - val_loss: 0.0350 -

In [35]:
model.save('asl_model')